In [76]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed
set_seed(3407)
import random

## Dataset

In [77]:
def l_z(length, num):
    num_str = str(num)
    if len(num_str) < length:
        return '0'*(length-len(num_str)) + num_str
    return num_str

def step_mul(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]

    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    if val_a < val_b:
        val_a, val_b = val_b, val_a
        a, b = b, a
    
    steps = [f"{l_z(length, val_b)}*{l_z(length, val_a)}="]
    steps_eq = []
    for i in range(length):
        a_i = a[i] * 10**(length-i-1)
        # a_i = str(a[i]) 
        steps.append(f"{l_z(length, val_b)}*{l_z(length, a_i)}+")
        steps_eq.append(l_z(2*length, a_i*val_b)+"+")
    
    steps[-1] = steps[-1][:-1] + "="
    steps_eq[-1] = steps_eq[-1][:-1] + "="

    steps.extend(steps_eq)
    steps.append(l_z(2*length, val_a*val_b))
    out = ''.join(steps)
    # out = list(out)
    return out
    # print(out)

def tokenize_string(string):
    out = []
    for char in string:
        if char == '+':
            out.append(10)
        elif char == '=':
            out.append(11)
        elif char == '*':
            out.append(12)
        else:
            out.append(int(char))
    return out


eq = step_mul(3)
print(eq)
tokenize_string(eq)


062*536=062*500+062*030+062*006=031000+001860+000372=033232


[0,
 6,
 2,
 12,
 5,
 3,
 6,
 11,
 0,
 6,
 2,
 12,
 5,
 0,
 0,
 10,
 0,
 6,
 2,
 12,
 0,
 3,
 0,
 10,
 0,
 6,
 2,
 12,
 0,
 0,
 6,
 11,
 0,
 3,
 1,
 0,
 0,
 0,
 10,
 0,
 0,
 1,
 8,
 6,
 0,
 10,
 0,
 0,
 0,
 3,
 7,
 2,
 11,
 0,
 3,
 3,
 2,
 3,
 2]

In [124]:
class MulDataset(Dataset):
    """ 
    Dataset for the Add problem. E.g. for problem length 3:
    12 + 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 3 4
    output: I I I I I 0 3 4 5
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 1000 # ...
    
    def get_vocab_size(self):
        return 13
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return len(tokenize_string(step_mul(self.length))) - 1

    def __getitem__(self, idx):
        while True:
            rai = tokenize_string(step_mul(self.length))
            h = hash(str(rai[:1+2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length] = -1
        return x, y

In [125]:
# print an example instance of the dataset
train_dataset = MulDataset('train')
test_dataset = MulDataset('test')
x, y = train_dataset[0]

print (x)
for a, b in zip(x,y):
    print(int(a),int(b))

tensor([ 2,  5,  1, 12,  8,  1,  2, 11,  2,  5,  1, 12,  8,  0,  0, 10,  2,  5,
         1, 12,  0,  1,  0, 10,  2,  5,  1, 12,  0,  0,  2, 11,  2,  0,  0,  8,
         0,  0, 10,  0,  0,  2,  5,  1,  0, 10,  0,  0,  0,  5,  0,  2, 11,  2,
         0,  3,  8,  1])
2 -1
5 -1
1 -1
12 -1
8 -1
1 -1
2 11
11 2
2 5
5 1
1 12
12 8
8 0
0 0
0 10
10 2
2 5
5 1
1 12
12 0
0 1
1 0
0 10
10 2
2 5
5 1
1 12
12 0
0 0
0 2
2 11
11 2
2 0
0 0
0 8
8 0
0 0
0 10
10 0
0 0
0 2
2 5
5 1
1 0
0 10
10 0
0 0
0 0
0 5
5 0
0 2
2 11
11 2
2 0
0 3
3 8
8 1
1 2


## Model

In [91]:
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-micro'
# model_config.model_type = 'gpt-nano'

model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.80M


In [ ]:
model.load_state_dict(torch.load('model_step_85.pth'))

In [92]:
print (model_config.n_head, model_config.n_layer, model_config.n_embd)

4 4 128


In [110]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 1e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 5000
train_config.num_workers = 0
# train_config.batch_size = 32
trainer = Trainer(train_config, model, train_dataset)

running on device cuda


In [112]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

iter_dt 46.56ms; iter 0: train loss 0.02889
iter_dt 46.96ms; iter 100: train loss 0.02667
iter_dt 45.86ms; iter 200: train loss 0.01901
iter_dt 46.42ms; iter 300: train loss 0.01788
iter_dt 47.72ms; iter 400: train loss 0.01980
iter_dt 46.57ms; iter 500: train loss 0.01923
iter_dt 45.33ms; iter 600: train loss 0.01809
iter_dt 45.67ms; iter 700: train loss 0.01690
iter_dt 45.45ms; iter 800: train loss 0.02092
iter_dt 45.99ms; iter 900: train loss 0.01830


KeyboardInterrupt: 

In [113]:
# now let's perform some evaluation
model.eval()
None

In [ ]:
def eval_add_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=50, num_workers=0, drop_last=False)
    #loader = DataLoader(dataset, batch_size=1, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)

        inp = x[:, :2*n+1]
        sol = y[:, -2*n:]
        
        cat = model.generate(inp, 52, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, -2*n:]      
        # print(cat[0])
        # print(inp[0])
        # print(sol[0])   
        # print(sol_candidate[0])
        correct = (sol == sol_candidate).all(1).cpu() 
        for i in range(x.size(0)):
            results.append(int(correct[i]))
    
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_add_split(trainer, 'train', max_batches=5)
    test_score  = eval_add_split(trainer, 'test',  max_batches=5)

In [115]:
torch.save(model.state_dict(), 'model_step_85.pth')